<a href="https://colab.research.google.com/github/mechamrit/StockAnalysis/blob/master/stock1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 1

[Using Pandas web Data Reader](https://pandas-datareader.readthedocs.io/en/latest/?source=post_page---------------------------) for getting the data from the Yahoo Finance. 
<br>For this we will use 1 Year of Google Data
<br> Importing Datetime Library to Work with the dates

In [1]:
## Import the essential Libraries
import pandas as pd
## https://pandas-datareader.readthedocs.io/en/latest/?source=post_page---------------------------b
import pandas_datareader.data as pdr
from datetime import datetime, timedelta 
import numpy as np

##datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])
start = datetime(2018 , 7 ,1)
end = datetime(2019, 7 , 1)

## Importing the Google finance
df1 = pdr.DataReader("GOOG", 'yahoo', start, end)
df1.tail(3)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-06-27,1087.099976,1075.290039,1084.000000,1076.010010,1004300,1076.010010
2019-06-28,1081.000000,1073.369995,1076.390015,1080.910034,1693200,1080.910034
2019-07-01,1107.579956,1093.703003,1098.000000,1097.949951,1436300,1097.949951


## Dropping the rows with Null Values(in Caif present)
## inplace _True will change  the Value in actual data Frame
df1.dropna(inplace=True)

2.Let us calculate the maximum, minimum and mean price for the last 90 days. We will use close price.

In [2]:
## We will Calculate the max , min , and mean with inbuilt functions

date_90 = end - timedelta(90)
new_db = df1[date_90 : end]

print('Max close Price', new_db.Close.max())
print('Min close Price', new_db.Close.min())
print('Mean close Price', new_db.Close.mean())

Max close Price 1287.5799560546875
Min close Price 1036.22998046875
Mean close Price 1152.2779347253224


3. Now we will calculate monthwise [VWAP](https://school.stockcharts.com/doku.php?id=technical_indicators:vwap_intraday)(Volume Weighted Average Price ) of the stock. Here we will use Pandas [groupBy()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) statement. Here we are calculating daily vwap.

In [3]:
## For group by we will add month and year column 

## Extracting year and month from the index
new_db['Month'] = pd.DatetimeIndex(new_db.index).month
new_db['Year'] = pd.DatetimeIndex(new_db.index).year

## Calulating VWAP
new_db['VWAP'] = (new_db.Close * new_db.Volume).cumsum() / new_db.Volume.cumsum()

## Now Grouping The Data
new_db1 = new_db.groupby(['Year' , 'Month']).mean()
new_db1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

High          Low  ...    Adj Close         VWAP
Year Month                            ...                          
2019 4      1231.842645  1218.398999  ...  1227.815997  1213.735834
     5      1161.281544  1140.388411  ...  1150.763178  1201.037532
     6      1091.008508  1072.817352  ...  1081.122504  1154.548145
     7      1107.579956  1093.703003  ...  1097.949951  1143.978869

[4 rows x 7 columns]

4. Let us write a function to calculate the average price over the last N days of the stock price data where N is a user defined parameter.

In [4]:
def avgPrice(N):
    delta = end - timedelta(days=N)
    db = df1[delta : end]
    print('Average Price is',db.Close.mean())
    return;
avgPrice(90)

Average Price is 1152.2779347253224


5. Write a second function to calculate the profit/loss percentage over the last N days.

In [6]:
def profLoss(N):
    delta = end - timedelta(days=N)
    db = df1[delta:end]
    print('Profit or Loss',round(db.Close.pct_change().mean(),3))
    
profLoss(90)

Profit or Loss -0.001


6. Let us add a column 'Day_Perc_Change' where the values are the daily change in percentages i.e. the percentage change between 2 consecutive day's closing prices.

In [7]:
df1['Day_perc_change'] = df1.Close.pct_change()
df1.head(3)


,High,Low,Open,Close,Volume,Adj Close,Day_perc_change
Date,,,,,,,
2018-07-02,1128.000000,1093.800049,1099.000000,1127.459961,1217300,1127.459961,NaN
2018-07-03,1135.819946,1100.020020,1135.819946,1102.890015,679000,1102.890015,-0.021792
2018-07-05,1127.500000,1108.479980,1110.530029,1124.270020,1066700,1124.270020,0.019385


In [0]:
## As we Can Observe that first row has NaN value
## So we will Drop the row or we can assign it any number or 0  so that it does not effect result

df1.dropna(inplace=True)


## or
# df1.loc[0,['Day_Perc_Change']] = 0

7. Let us add another column 'Trend' whose values are:<br>
<br>
   - 'Slight or No change' for 'Day_Perc_Change' in between -0.5 and 0.5 <br>
   - 'Slight positive' for 'Day_Perc_Change' in between 0.5 and 1<br>
   - 'Slight negative' for 'Day_Perc_Change' in between -0.5 and -1 <br>
   - 'Positive' for 'Day_Perc_Change' in between 1 and 3 <br>
   - 'Negative' for 'Day_Perc_Change' in between -1 and -3 <br>
   - 'Among top gainers' for 'Day_Perc_Change' in between 3 and 7 <br>
   - 'Among top losers' for 'Day_Perc_Change' in between -3 and -7 <br>
   - 'Bull run' for 'Day_Perc_Change' >7 <br>
   - 'Bear drop' for 'Day_Perc_Change' <-7 <br><br>


** Simple if else do not work properly so we will use [np.select](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html)

In [9]:
conditions = [(df1.Day_perc_change>-0.5) & (df1.Day_perc_change<0.5) , 
            (df1.Day_perc_change < 1) & (df1.Day_perc_change >0.5),
            (df1.Day_perc_change>1) & (df1.Day_perc_change<3),
            (df1.Day_perc_change<-0.5) & (df1.Day_perc_change>-1),
            (df1.Day_perc_change<-1) & (df1.Day_perc_change>-3),
            (df1.Day_perc_change>3) & (df1.Day_perc_change<7),
            (df1.Day_perc_change<-3) & (df1.Day_perc_change>-7),
            (df1.Day_perc_change>7),
            (df1.Day_perc_change<-7)
]


choices =[
            "Slight or No Change",
            'Slight Positive',
            'Positive',
            'Slight Negative',
            'Negative',
            'Among Top Gainers',
            'Among Top Losers',
            'Bull run',
            'Bear Drop'
    
]

df1['Trend'] = np.select(conditions,choices)
df1.head(10)

,High,Low,Open,Close,Volume,Adj Close,Day_perc_change,Trend
Date,,,,,,,,
2018-07-03,1135.819946,1100.020020,1135.819946,1102.890015,679000,1102.890015,-0.021792,Slight or No Change
2018-07-05,1127.500000,1108.479980,1110.530029,1124.270020,1066700,1124.270020,0.019385,Slight or No Change
2018-07-06,1140.930054,1120.737061,1123.579956,1140.170044,996100,1140.170044,0.014143,Slight or No Change
2018-07-09,1154.670044,1143.420044,1148.479980,1154.050049,909000,1154.050049,0.012174,Slight or No Change
2018-07-10,1159.589966,1149.589966,1156.979980,1152.839966,798400,1152.839966,-0.001049,Slight or No Change
2018-07-11,1164.290039,1141.000000,1144.589966,1153.900024,1120000,1153.900024,0.000920,Slight or No Change
2018-07-12,1184.410034,1155.935059,1159.890015,1183.479980,1251900,1183.479980,0.025635,Slight or No Change
2018-07-13,1195.416992,1180.000000,1185.000000,1188.819946,1221900,1188.819946,0.004512,Slight or No Change
2018-07-16,1191.000000,1179.280029,1189.390015,1183.859985,1055700,1183.859985,-0.004172,Slight or No Change


8. Lets find the Average and Median values of Column 'Volume' for each of the types of 'Trend'. 
<br>Here we will first use the Group by on Trend and then calculate average and median values of the column.

In [10]:
print('The Average Value is',df1.groupby(df1.Trend)['Volume'].mean(),'\n')
print('The Median Value is' , df1.groupby(df1.Trend)['Volume'].median())

## Here we got only one trend as stock prices of google is quite stable 
## Therfore trend is only Slight or No change

The Average Value is Trend
Slight or No Change    1607370.8
Name: Volume, dtype: float64 

The Median Value is Trend
Slight or No Change    1436100
Name: Volume, dtype: int64


In [0]:
## Now let us Save the Data Frame
df1.to_csv('google.csv')

** Check the Working Directory